In [1]:
!pip install tensorflow-addons


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ZeroPadding2D,Activation, Conv2D, Dropout, MaxPooling2D, Flatten, Dense, Lambda

In [3]:
#! pip install -q kaggle

In [4]:
#! mkdir ~/.kaggle

In [5]:
#! cp kaggle.json ~/.kaggle/

In [6]:
#! chmod 600 ~/.kaggle/kaggle.json

In [7]:
#!kaggle datasets download -d hereisburak/pins-face-recognition

In [8]:
#! unzip pins-face-recognition.zip -d pins-face-recognition

In [9]:
directory = "/home/batman/Desktop/face_recog/train/content/pins-face-recognition/Training"

In [10]:
IMG_SHAPE = (224,224,3)
batch_size = 64
EPOCHS = 50
base_output = "/content/output"
MODEL_PATH = os.path.sep.join([base_output, "siamese_model"])
PLOT_PATH = os.path.sep.join([base_output, "plot.png"])

In [11]:
def build_siamese_model(inputShape, embeddingDim=48):
    # specify the inputs for the feature extractor network
    inputs = Input(inputShape)
    classifier= Sequential()
    classifier.add(Convolution2D(32, kernel_size=(3, 3), strides=(1, 1), input_shape=(224,224,3), activation='relu'))
    classifier.add(Convolution2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(MaxPool2D(pool_size=(2,2)))

    classifier.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(Convolution2D(64, kernel_size=(3,3), strides=(1, 1), activation='relu'))
    classifier.add(MaxPool2D(pool_size=(2,2)))
    
    classifier.add(Convolution2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(Convolution2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(MaxPool2D(pool_size=(2,2)))
    
    classifier.add(Convolution2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(Convolution2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(Convolution2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(MaxPool2D(pool_size=(2,2)))

    classifier.add(Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(MaxPool2D(pool_size=(2,2)))

    classifier.add(Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(Convolution2D(512, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    classifier.add(MaxPool2D(pool_size=(2,2)))

    classifier.add(Flatten())
    
    classifier.add(Dense(4096, activation='relu'))
    classifier.add(keras.layers.Dropout(0.5))
    classifier.add(Dense(4096, activation='relu'))
    classifier.add(keras.layers.Dropout(0.5))
    classifier.add(Dense(OutputNeurons, activation='softmax'))
    # prepare the final outputs
    classifier.add(GlobalAveragePooling2D())
    output = classifier.add(Dense(embeddingDim))
    
    model = Model(inputs, outputs)
    
    return model

In [12]:
def make_pairs(images, labels):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImages = []
    pairLabels = []
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    
    unique_label = np.unique(labels)
    idx = {}
    print(len(unique_label))
    for i in range(len(unique_label)):
        ind = []
        for j in range(len(labels)):
            if labels[j] == unique_label[i]:
                ind.append(j)
        idx[unique_label[i]] = ind
    # loop over all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current
        # iteration
        currentImage = images[idxA]
        label = labels[idxA]
        # randomly pick an image that belongs to the *same* class
        # label
        temp_lis = idx[label]
        idxB = random.choice(temp_lis)
        posImage = images[idxB]
        # prepare a positive pair and update the images and labels
        # lists, respectively
        pairImages.append([currentImage, posImage])
        pairLabels.append([1])
        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        negIdx = np.where(labels != label)[0]
        negImage = images[np.random.choice(negIdx)]
        # prepare a negative pair of images and update our lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([0])
    # return a 2-tuple of our image pairs and labels
    return (np.array(pairImages), np.array(pairLabels))

In [13]:
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
        keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [14]:
# images = []
# label = []
# for i in tqdm(os.listdir(directory)):
#     lbl = i[5:]
#     p = os.path.join(directory,i)
#     print(i)
#     for j in tqdm(os.listdir(p)):
#         print(j)
#         img= cv2.imread(os.path.join(p, j))
#         img2 = cv2.resize(img,(224,224))
#         images+=img2
#         label += lbl

In [15]:
# def detect_face(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#     faces = face_cascade.detectMultiScale(gray, 1.1, 4)
#     # Draw rectangle around the faces and crop the faces
#     for (x, y, w, h) in faces:
#         cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
#         faces = image[y:y + h, x:x + w]
#         #cv2.imshow("face",faces)
#     #print(faces)
# #     cv2.imshow("face", faces)
# #     cv2.waitKey(0)
# #     cv2.destroyAllWindows()
#     return faces

In [16]:
# images=[]
# label=[]
# count=0
# for file in tqdm(os.listdir(directory)):
#     if file == 'Untitled.ipynb' or file == '.ipynb_checkpoints':
#         continue
        
#     path=os.path.join(directory,file)
#     name = file[5:]
#     for im in os.listdir(path):
#         image=cv2.imread(os.path.join(path,im))
#         #image2=mtcnn_detector(image)
#         image2 = detect_face(image)
# #         if type(image2) == "<class 'tuple'>":
# #             continue
        
#         if image2 == ():
#             continue
#         #if image2.shape[0]>10 and image2.shape[1]>10:
#         image3=cv2.resize(image2,(224,224))
#         # image3 = image3/255
#         images+=[image3]
#         label+=[name]

In [17]:
#!unzip train.zip -d train

In [18]:
images = []
label = []
for file in tqdm(os.listdir(directory)):
    path = directory + "/" + file
    for img in os.listdir(path):
        p = path + "/" + img
        image = cv2.imread(p)
        images.append(image)
        label.append(file[5:])

100%|██████████| 105/105 [00:24<00:00,  4.30it/s]


In [19]:
print(len(images))
print(len(label))


17514
17514


In [20]:
n=len(images)
print(n)
D=list(range(n))
random.seed(2021)
random.shuffle(D)

17514


In [ ]:
trainY=np.array(label)[D[0:(n//4)*3]]
testY=np.array(label)[D[(n//4)*3:]]
trainX=np.array(images)[D[0:(n//4)*3]]
testX=np.array(images)[D[(n//4)*3:]]

In [ ]:

# testX = testX / 255.0
# add a channel dimension to the images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
# prepare the positive and negative pairs
print("[INFO] preparing positive and negative pairs...")
(pairTrain, labelTrain) = make_pairs(trainX, trainY)
(pairTest, labelTest) = make_pairs(testX, testY)

In [ ]:
print(pairTrain)

In [ ]:
print("[INFO] building siamese network...")
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)
featureExtractor = build_siamese_model(IMG_SHAPE)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

In [ ]:
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

In [ ]:
# compile the model
print("[INFO] compiling model...")
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# train the model
print("[INFO] training model...")
history = model.fit(
    [pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
    validation_data=([pairTest[:, 0], pairTest[:, 1]], labelTest[:]),
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS)

In [ ]:
def plot_training(H, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.history["loss"], label="train_loss")
    plt.plot(H.history["val_loss"], label="val_loss")
    plt.plot(H.history["accuracy"], label="train_acc")
    plt.plot(H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

In [ ]:
# serialize the model to disk
print("[INFO] saving siamese model...")
model.save(MODEL_PATH)
# plot the training history
print("[INFO] plotting training history...")
plot_training(history, PLOT_PATH)